<a href="https://colab.research.google.com/github/ymoslem/file-converters/blob/main/TMX2MT/TMX2MT-new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converting a TMX translation memory for MT training

* Convert to Moses format
 * One text file per language, one sentence per line
* Convert to GPT fine-tuning format
 * JSON lines (JSONL)

# Converting XML to JSON

* Initial step that is important for all formats

In [1]:
!pip3 install xmltodict -q

In [2]:
!wget -qq --show-progress show http://optima.jrc.it/Resources/ECDC-TM/ECDC-TM.zip

ECDC-TM.zip         100%[===================>]   4.09M  5.39MB/s    in 0.8s    


In [3]:
!unzip -q -n ECDC-TM.zip
%cd ECDC-TM
!ls

/content/ECDC-TM
2012_10_ECDC-TM-Statistics.pdf	  ECDC.tmx
2012_10_Terms-of-Use_ECDC-TM.pdf  Readme_CreateLanguagePair.txt
CreateLanguagePair.jar		  Readme_ECDC-TM.txt
ECDC-domains.xlsx		  tmx14.dtd


In [4]:
# Convert XML to JSON

import json
import xmltodict

tmx_file = "ECDC.tmx"

with open("ECDC.tmx") as xml_file:
  data_dict = xmltodict.parse(xml_file.read())

In [5]:
# Explore the TMX file

for tu in data_dict["tmx"]["body"]["tu"][:2]:
  print("\n")
  for unit in tu["tuv"]:
    language = unit["@xml:lang"]
    segment = unit["seg"]
    print(f"{language}: {segment}")



EN: Vaccination against hepatitis C is not yet available.
BG: Засега няма ваксина срещу хепатит С.
CS: Očkování proti hepatitidě C zatím není k dispozici.
DA: Der findes endnu ingen vaccination mod hepatitis C.
DE: Es gibt derzeit noch keinen Impfstoff gegen Hepatitis C.
EL: Επί του παρόντος δεν διατίθεται εμβόλιο έναντι της ηπατίτιδας C.
ES: Todavía no hay ninguna vacuna contra la hepatitis C.
ET: C-hepatiidi vastast vaktsiini veel ei ole.
FI: Hepatiitti C:hen ei ole vielä rokotetta.
FR: Aucune vaccination contre l’hépatite C n’est encore disponible.
GA: Níl vacsaíniú ar fáil fós i gcoinne heipitítis C.
HU: A hepatitisz C ellen nincs még védőoltás.
IS: Bólusetning gegn lifrarbólgu C er enn ekki til.
IT: Non c'è ancora un vaccino per l'epatite C.
LT: Vakcinos nuo hepatito C dar nėra.
LV: Pagaidām pret hepatītu C nav iespējams vakcinēties.
MT: It-tilqim kontra l-epatite Ċ għadu mhux disponibbli.
NL: Er is nog geen vaccin tegen hepatitis C beschikbaar.
NO: Det finnes foreløpig ingen va

In [6]:
# Create a list of translation units

data = [tu["tuv"] for tu in data_dict["tmx"]["body"]["tu"]]
print(data[0:4])

[[{'@xml:lang': 'EN', 'seg': 'Vaccination against hepatitis C is not yet available.'}, {'@xml:lang': 'BG', 'seg': 'Засега няма ваксина срещу хепатит С.'}, {'@xml:lang': 'CS', 'seg': 'Očkování proti hepatitidě C zatím není k\xa0dispozici.'}, {'@xml:lang': 'DA', 'seg': 'Der findes endnu ingen vaccination mod hepatitis C.'}, {'@xml:lang': 'DE', 'seg': 'Es gibt derzeit noch keinen Impfstoff gegen Hepatitis C.'}, {'@xml:lang': 'EL', 'seg': 'Επί του παρόντος δεν διατίθεται εμβόλιο έναντι της ηπατίτιδας C.'}, {'@xml:lang': 'ES', 'seg': 'Todavía no hay ninguna vacuna contra la hepatitis C.'}, {'@xml:lang': 'ET', 'seg': 'C-hepatiidi vastast vaktsiini veel ei ole.'}, {'@xml:lang': 'FI', 'seg': 'Hepatiitti C:hen ei ole vielä rokotetta.'}, {'@xml:lang': 'FR', 'seg': 'Aucune vaccination contre l’hépatite C n’est encore disponible.'}, {'@xml:lang': 'GA', 'seg': 'Níl vacsaíniú ar fáil fós i gcoinne heipitítis C.'}, {'@xml:lang': 'HU', 'seg': 'A hepatitisz C ellen nincs még védőoltás.'}, {'@xml:lang':

# Convert to Moses format
* convert a TM to Moser format (txt file, with one segment per line)
* use Polars - recommended

In [8]:
!pip3 install polars -q

In [7]:
#@title Using Pandas (slower than Polars)


# Remove this line to run the cell
%%script false --no-raise-error


import pandas as pd

df_list = []
for idx, group in enumerate(data):
    df_temp = pd.DataFrame(group)
    df_temp['group'] = idx
    df_list.append(df_temp)

# Using pivot_table to handle potential duplicates
df_pivot_pd = pd.concat(df_list).pivot_table(index='group',
                                             columns='@xml:lang',
                                             values='seg',
                                             aggfunc='first').reset_index(drop=True)

# Repace nan
df_pivot_pd = df_pivot_pd.fillna("NA")

# [Optional] Select only a few languages
df_pivot_pd = df_pivot_pd[['EN', 'BG', 'CS', 'GA']]

df_pivot_pd.head(10)

In [9]:
#@title Using Polars - recommended

import polars as pl

# Convert the nested list into a DataFrame
df = pl.DataFrame({
    'lang': [entry['@xml:lang'] for sublist in data for entry in sublist],
    'seg': [entry['seg'] for sublist in data for entry in sublist]
})

# Create a unique index column to keep track of each group of translations
df = df.with_columns([(pl.col("lang") == "EN").cumsum().alias("index")])

# Pivot the data
df = df.pivot(index="index", columns="lang", values="seg", aggregate_function="first")

# Fill missing translations
# Later you will have to filter out empty segments
# df = df.fill_null("NA")

# Remove newlines "\n" from translations
df = df.with_columns(pl.col("*").cast(pl.Utf8).str.strip().str.replace_all("\n", " "))

df.head(10)

index,EN,BG,CS,DA,DE,EL,ES,ET,FI,FR,GA,HU,IS,IT,LT,LV,MT,NL,NO,PL,PT,RO,SK,SL,SV
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""1""","""Vaccination ag…","""Засега няма ва…","""Očkování proti…","""Der findes end…","""Es gibt derzei…","""Επί του παρόντ…","""Todavía no hay…","""C-hepatiidi va…","""Hepatiitti C:h…","""Aucune vaccina…","""Níl vacsaíniú …","""A hepatitisz C…","""Bólusetning ge…","""Non c'è ancora…","""Vakcinos nuo h…","""Pagaidām pret …","""It-tilqim kont…","""Er is nog geen…","""Det finnes for…","""Niestety nadal…","""De momento, nã…","""Nu există încă…","""Proti hepatití…","""Cepiva proti h…","""Det finns ännu…"
"""2""","""HIV infection""","""Инфекция с чов…","""Infekce HIV""","""HIV-infektion""","""HIV-Infektion""","""Λοίμωξη από το…","""VIH, infección…","""HIV-nakkus""","""HIV-infektio""","""Infection à VI…","""Ionfhabhtú VEI…","""HIV fertőzés""","""HIV smitun""","""Infezione da H…","""ŽIV infekcija""","""HIV infekcija""","""Infezzjoni HIV…","""Hiv-infectie""","""HIV-infeksjon""","""Zakażenie wiru…","""Infecção por V…","""Infecţia cu HI…","""Infekcia HIV""","""Okužba s HIV""","""HIV-infektion"""
"""3""","""The human immu…","""Човешкият имун…","""Virus lidské i…","""Hiv (humant im…","""Das humane Imm…","""Ο ιός της ανοσ…","""La infección p…","""Inimese immuun…","""Ihmisen immuun…","""L’infection pa…","""Tá an víreas e…","""A humán immunh…","""Alnæmisveira í…","""Il virus dell'…","""Žmogaus imunod…","""Cilvēka imūnde…","""Il-virus tal-i…","""Het humaan imm…","""Humant immunsv…","""Ludzki wirus u…","""O vírus da imu…","""Infecţia cu vi…","""Vírus ľudskej …","""Virus humane i…","""Humant immunbr…"
"""4""","""It is an infec…","""Тази инфекция …","""Jde o infekci …","""Hiv er forbund…","""Eine Infektion…","""Πρόκειται για …","""Se trata de un…","""See on nakkus,…","""HIV-infektio o…","""Cette infectio…","""Is ionfhabhtú …","""Ez egy olyan s…","""HIV smitun vel…","""Si tratta di u…","""Tai infekcija,…","""Tā ir infekcij…","""Dan huwa infez…","""Het is een inf…","""Infeksjonen as…","""Jest to zakaże…","""Trata-se de um…","""Este o infecţi…","""Je to infekcia…","""Gre za okužbo,…","""HIV-infektion …"
"""5""","""HIV is a virus…","""ХИВ атакува им…","""HIV je virus, …","""HIV er et viru…","""HIV ist ein Vi…","""O HIV είναι έν…","""El VIH es un v…","""HIV on viirus,…","""HIV on virus, …","""Le VIH est un …","""Is víreas é VE…","""A HIV egy víru…","""HIV er veira s…","""L’HIV è un vir…","""ŽIV yra virusa…","""HIV ir vīruss,…","""L-HIV hu virus…","""Hiv is een vir…","""HIV er et viru…","""HIV jest wirus…","""O VIH é um vír…","""HIV este un vi…","""HIV je vírus, …","""HIV je virus, …","""HIV är ett vir…"
"""6""","""HIV is spread …","""Разпространява…","""HIV se šíří po…","""Hiv spredes ve…","""HIV wird übert…","""Ο HIV μεταδίδε…","""El VIH se cont…","""HIV levib seks…","""HIV leviää suk…","""Le VIH se tran…",null,"""A HIV fertőzöt…","""HIV dreifist m…","""Si trasmette a…","""ŽIV plinta per…","""HIV izplatās d…","""L-HIV jinxtere…","""Hiv wordt vers…","""HIV spres gjen…","""Wirus HIV prze…","""O VIH transmit…","""HIV se transmi…","""HIV sa šíri po…","""HIV se širi s …","""HIV sprids gen…"
"""7""","""Babies born to…","""Бебетата на ин…","""Děti narozené …","""Spædbørn født …","""Babys von HIV-…","""Τα βρέφη που γ…","""Los niños naci…","""HIV-nakkusega …","""HIV-infektion …","""Les enfants de…","""D'fhéadfadh le…","""A HIV fertőzöt…","""Börn HIV smita…","""I bambini che …","""Kūdikiai, pagi…","""Bērni, kuri pi…","""Trabi li jitwi…","""Baby's van met…","""Barn som fødes…","""Dzieci kobiet …","""Os filhos de m…","""Copiii născuţi…","""Deti narodené …","""Dojenčki, ki s…","""Barn som föds …"
"""8""","""The end-stage …","""Крайната фаза …","""Konečné stadiu…","""Slutstadiet i …","""Das Endstadium…","""Το τελικό στάδ…","""La fase final …","""Nakkuse lõppst…","""Infektion lopp…","""Le stade final…","""Tarlaíonn stai…","""A fertőzés vég…","""Lokastigið eyð…","""Lo stadio fina…"

In [10]:
# Save each column to a text file.

print("Saving files...")

for lang in df.columns[1:]:
  lang_column = df.select(lang)
  lang_column.write_csv("ECDC."+lang,
                        has_header=False,
                        null_value="NA",
                        separator="=")

Saving files...


In [11]:
# Make sure all the files have the same number lines
!wc -l ECDC.*

    3919 ECDC.BG
    3919 ECDC.CS
    3919 ECDC.DA
    3919 ECDC.DE
    3919 ECDC.EL
    3919 ECDC.EN
    3919 ECDC.ES
    3919 ECDC.ET
    3919 ECDC.FI
    3919 ECDC.FR
    3919 ECDC.GA
    3919 ECDC.HU
    3919 ECDC.IS
    3919 ECDC.IT
    3919 ECDC.LT
    3919 ECDC.LV
    3919 ECDC.MT
    3919 ECDC.NL
    3919 ECDC.NO
    3919 ECDC.PL
    3919 ECDC.PT
    3919 ECDC.RO
    3919 ECDC.SK
    3919 ECDC.SL
    3919 ECDC.SV
  231650 ECDC.tmx
  329625 total


In [12]:
!head ECDC.EN
!echo
!head ECDC.GA
!echo
!head ECDC.FR

Vaccination against hepatitis C is not yet available.
HIV infection
The human immunodeficiency virus (HIV) remains one of the most important communicable diseases in Europe.
It is an infection associated with serious disease, persistently high costs of treatment and care, significant number of deaths and shortened life expectancy.
HIV is a virus, which attacks the immune system and causes a lifelong severe illness with a long incubation period.
HIV is spread by sexual contact with an infected person, by sharing needles or syringes (primarily for drug injection) with someone who is infected, or, less commonly (and now very rarely in countries where blood is screened for HIV antibodies), through transfusions of infected blood or blood clotting factors.
Babies born to HIV-infected women may become infected before or during birth or through breast-feeding.
The end-stage of the infection, acquired immunodeficiency syndrome (AIDS), results from the destruction of the immune system.
AIDS is d

# Convert to GPT fine-tuning format

* You can find more details about the fine-tuning process and supported models [here](https://platform.openai.com/docs/guides/fine-tuning).

```python
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
...
```

In [13]:
# Let's get English and German columns
en_de_df = df.select("EN", "DE")

# Rename columns to "prompt" and "completion"
en_de_df = en_de_df.rename({"EN": "prompt"})
en_de_df = en_de_df.rename({"DE": "completion"})

# [Optional] Add language names to the prompt, recommended for tranlsation tasks
en_de_df = en_de_df.with_columns(en_de_df['prompt'].apply(lambda col: "English: " + col + "\nGerman:"))


pl.Config.set_fmt_str_lengths(200)
en_de_df.head(3)

prompt,completion
str,str
"""English: Vaccination against hepatitis C is not yet available. German:""","""Es gibt derzeit noch keinen Impfstoff gegen Hepatitis C."""
"""English: HIV infection German:""","""HIV-Infektion"""
"""English: The human immunodeficiency virus (HIV) remains one of the most important communicable diseases in Europe. German:""","""Das humane Immundefizienz-Virus HIV gehört weiterhin zu den bedeutendsten Infektionserregern in Europa."""


In [14]:
# Remove null rows

print("Segments:", en_de_df.shape[0])
en_de_df = en_de_df.drop_nulls()
print("Segments:", en_de_df.shape[0])

Segments: 3919
Segments: 2560


In [15]:
# Convert the dataframe to JSON lines (JSONL)

json_lines = en_de_df.to_dicts()

print(*json_lines[:3], sep="\n")

{'prompt': 'English: Vaccination against hepatitis C is not yet available.\nGerman:', 'completion': 'Es gibt derzeit noch keinen Impfstoff gegen Hepatitis C.'}
{'prompt': 'English: HIV infection\nGerman:', 'completion': 'HIV-Infektion'}
{'prompt': 'English: The human immunodeficiency virus (HIV) remains one of the most important communicable diseases in Europe.\nGerman:', 'completion': 'Das humane Immundefizienz-Virus HIV gehört weiterhin zu den bedeutendsten Infektionserregern in Europa.'}


In [16]:
len(json_lines)

2560

In [17]:
import json

with open("en-de.jsonl", "w+") as jsonl:
  for line in json_lines:
    json.dump(line, jsonl)
    jsonl.write("\n")

In [18]:
!wc -l "en-de.jsonl"

2560 en-de.jsonl


In [19]:
!head -n 3 "en-de.jsonl"

{"prompt": "English: Vaccination against hepatitis C is not yet available.\nGerman:", "completion": "Es gibt derzeit noch keinen Impfstoff gegen Hepatitis C."}
{"prompt": "English: HIV infection\nGerman:", "completion": "HIV-Infektion"}
{"prompt": "English: The human immunodeficiency virus (HIV) remains one of the most important communicable diseases in Europe.\nGerman:", "completion": "Das humane Immundefizienz-Virus HIV geh\u00f6rt weiterhin zu den bedeutendsten Infektionserregern in Europa."}


# Creating a multilingual dataset

## Simple example

In [46]:
%%time

# Example with Pandas

import pandas as pd

data_example = {'en': ['hello', 'cat', 'dog'],
        'de': ['hallo', 'Katze', 'Hund'],
        'fr': ['bonjour', 'chat', 'chien']}

df_example = pd.DataFrame(data_example)

translations = []

# Get the list of languages
languages = df_example.columns

for from_lang in languages:
  for to_lang in languages:
    if from_lang != to_lang:  # Avoid adding translations within the same language
      from_words = df_example[from_lang]
      to_words = df_example[to_lang]
      for from_word, to_word in zip(from_words, to_words):
        translations.append({'from_word': from_word, 'to_word': to_word})

result_df = pd.DataFrame(translations)
#result_df

CPU times: user 1.73 ms, sys: 0 ns, total: 1.73 ms
Wall time: 1.74 ms


In [41]:
%%time

import pandas as pd

data_example = {
    'en': ['hello', 'cat', 'dog'],
    'de': ['hallo', 'Katze', 'Hund'],
    'fr': ['bonjour', 'chat', 'chien']
}

df_example = pd.DataFrame(data_example)
languages = df_example.columns

translations = []

for from_lang in languages:
    for to_lang in languages:
        if from_lang != to_lang:
            translation_pairs = list(zip(df_example[from_lang], df_example[to_lang]))
            translations.extend([{'from_word': from_word, 'to_word': to_word} for from_word, to_word in translation_pairs])

result_df = pd.DataFrame(translations)

# result_df

CPU times: user 1.92 ms, sys: 0 ns, total: 1.92 ms
Wall time: 4.3 ms


In [49]:
%%time

# Example with Polars

import polars as pl

data_example = {'en': ['hello', 'cat', 'dog'],
        'de': ['hallo', 'Katze', 'Hund'],
        'fr': ['bonjour', 'chat', 'chien']}

df_example = pl.DataFrame(data_example)

translations = []

# Get the list of languages
languages = df_example.columns

for from_lang in languages:
  for to_lang in languages:
    if from_lang != to_lang:
      from_words = df_example[from_lang]
      to_words = df_example[to_lang]
      translations.extend(list(zip(from_words, to_words)))

result_data = {'from_word': [t[0] for t in translations], 'to_word': [t[1] for t in translations]}
result_df = pl.DataFrame(result_data)
# result_df

CPU times: user 845 µs, sys: 0 ns, total: 845 µs
Wall time: 857 µs


## Apply to the dataset

In [22]:
# Apply to 3 languages, for example

en_de_ga_df = df.select("EN", "DE", "GA")

# If you create for the whole df, remove index
# whole_df = df.select(pl.exclude("index"))

print(en_de_ga_df.shape)
en_de_ga_df.head()

(3919, 3)


EN,DE,GA
str,str,str
"""Vaccination against hepatitis C is not yet available.""","""Es gibt derzeit noch keinen Impfstoff gegen Hepatitis C.""","""Níl vacsaíniú ar fáil fós i gcoinne heipitítis C."""
"""HIV infection""","""HIV-Infektion""","""Ionfhabhtú VEID"""
"""The human immunodeficiency virus (HIV) remains one of the most important communicable diseases in Europe.""","""Das humane Immundefizienz-Virus HIV gehört weiterhin zu den bedeutendsten Infektionserregern in Europa.""","""Tá an víreas easpa imdhíonachta daonna (VEID) fós ar cheann de na galair thógálacha is tábhachtaí san Eoraip."""
"""It is an infection associated with serious disease, persistently high costs of treatment and care, significant number of deaths and shortened life expectancy.""","""Eine Infektion mit diesem Virus führt zu ernsthaften Erkrankungen, dauerhaft hohen Behandlungs- und Pflegekosten, einer hohen Zahl von Todesfällen und einer verkürzten Lebenserwartung.""","""Is ionfhabhtú é a mbaineann galar tromchúiseach leis, chomh maith le costais cóireála agus cúraim síorarda, líon suntasach básanna agus ionchas saoil níos giorra."""
"""HIV is a virus, which attacks the immune system and causes a lifelong severe illness with a long incubation period.""","""HIV ist ein Virus, das das Immunsystem angreift und nach einer langen Inkubationszeit eine lebenslange schwere Krankheit hervorruft.""","""Is víreas é VEID, a dhéanann ionsaí ar an gcóras imdhíonachta agus a chruthaíonn géarbhreoiteacht ar feadh an tsaoil le tréimhse ghoir fhada."""


In [23]:
translations = []

# Get the list of languages
languages = en_de_ga_df.columns

for from_lang in languages:
  for to_lang in languages:
    if from_lang != to_lang:
      from_words = en_de_ga_df[from_lang]
      to_words = en_de_ga_df[to_lang]
      translations.extend(list(zip(from_words, to_words)))

en_de_ga_result_data = {'Source': [t[0] for t in translations], 'Target': [t[1] for t in translations]}
en_de_ga_result_df = pl.DataFrame(en_de_ga_result_data)

# N of langs (3) * N of langs - 1 (2) * N of rows (3919)
print(en_de_ga_result_df.shape)  # 3919 * 6 translation pairs

# Drop null values
en_de_ga_result_df = en_de_ga_result_df.drop_nulls()
print(en_de_ga_result_df.shape)

pl.Config.set_fmt_str_lengths(200)
en_de_ga_result_df.head(10)

(23514, 2)
(10324, 2)


Source,Target
str,str
"""Vaccination against hepatitis C is not yet available.""","""Es gibt derzeit noch keinen Impfstoff gegen Hepatitis C."""
"""HIV infection""","""HIV-Infektion"""
"""The human immunodeficiency virus (HIV) remains one of the most important communicable diseases in Europe.""","""Das humane Immundefizienz-Virus HIV gehört weiterhin zu den bedeutendsten Infektionserregern in Europa."""
"""It is an infection associated with serious disease, persistently high costs of treatment and care, significant number of deaths and shortened life expectancy.""","""Eine Infektion mit diesem Virus führt zu ernsthaften Erkrankungen, dauerhaft hohen Behandlungs- und Pflegekosten, einer hohen Zahl von Todesfällen und einer verkürzten Lebenserwartung."""
"""HIV is a virus, which attacks the immune system and causes a lifelong severe illness with a long incubation period.""","""HIV ist ein Virus, das das Immunsystem angreift und nach einer langen Inkubationszeit eine lebenslange schwere Krankheit hervorruft."""
"""HIV is spread by sexual contact with an infected person, by sharing needles or syringes (primarily for drug injection) with someone who is infected, or, less commonly (and now very rarely in countrie…","""HIV wird übertragen durch den sexuellen Kontakt zu einer infizierten Person, die gemeinsame Benutzung von Nadeln oder Spritzen (vor allem beim Spritzen von Drogen) mit einer infizierten Person oder, …"
"""Babies born to HIV-infected women may become infected before or during birth or through breast-feeding.""","""Babys von HIV-infizierten Müttern können vor oder während der Geburt oder beim Stillen infiziert werden."""
"""The end-stage of the infection, acquired immunodeficiency syndrome (AIDS), results from the destruction of the immune system.""","""Das Endstadium der Infektion ist die AIDS-Krankheit, ein erworbenes Immundefektsyndrom, das durch die Zerstörung des Immunsystems entsteht."""
"""AIDS is defined by the presence of one or more “opportunistic” illnesses (other illnesses due to decreased immunity).""","""AIDS ist definiert als das Vorhandensein einer oder mehrerer opportunistischer Krankheiten, das heißt Krankheiten, die sich aufgrund des geschwächten Immunsystems entwickeln konnten."""
